In [ ]:

from google.colab import drive
drive.mount('/content/drive')


!sudo apt-get install -y tesseract-ocr poppler-utils
!pip install -q transformers gradio torch accelerate Pillow PyPDF2 pdf2image pytesseract datasets "diffusers[torch]" moviepy


Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 3s (1,898 kB/s)
debconf: unable to

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `personal` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `personal`

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

!sudo apt-get install -y tesseract-ocr poppler-utils
!pip install -q transformers gradio torch accelerate Pillow PyPDF2 pdf2image pytesseract datasets "diffusers[torch]" moviepy opencv-python-headless

import re
import json
import os
import pytesseract
from PIL import Image
from transformers import pipeline, AutoTokenizer


!huggingface-cli login


model_name = "meta-llama/Llama-3.1-8B"


tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    device=0  # Use -1 for CPU if needed
)


def extract_ocr_text(image_path):
    """
    Opens an image and extracts text using pytesseract.
    """
    img = Image.open(image_path).convert("RGB")
    raw_text = pytesseract.image_to_string(img)
    cleaned_text = re.sub(r'\s+', ' ', raw_text).strip()
    return cleaned_text

def summarize_text(text):
    """
    Generates a summary for the provided text using the default generation settings.
    The prompt instructs the model to summarize the educational content.
    """
    prompt = f"Summarize the following educational content clearly and concisely:\n\n{text}\n\nSummary:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    summary_output = summarizer(
        prompt,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    generated = summary_output[0]['generated_text']
    if "Summary:" in generated:
        generated = generated.split("Summary:")[-1]
    return generated.strip()

def annotate_page(image_path, pdf_name, page_number):
    """
    Extracts OCR text from an image page and generates a refined annotation.
    """
    ocr_text = extract_ocr_text(image_path)
    if not ocr_text:
        return {
            "pdf_name": pdf_name,
            "page_number": page_number,
            "image_path": image_path,
            "ocr_text": "No text found",
            "annotation_text": "No annotation generated"
        }
    annotation = summarize_text(ocr_text)
    return {
        "pdf_name": pdf_name,
        "page_number": page_number,
        "image_path": image_path,
        "ocr_text": ocr_text,
        "annotation_text": annotation
    }


example_annotation = annotate_page("/content/drive/MyDrive/ScienceImages/jesc101_page_1.png", "jesc101.pdf", 1)
print(json.dumps(example_annotation, indent=4))


Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 3s (1,879 kB/s)
debconf: unable to

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Device set to use cuda:0


{
    "pdf_name": "jesc101.pdf",
    "page_number": 1,
    "image_path": "/content/drive/MyDrive/ScienceImages/jesc101_page_1.png",
    "ocr_text": "\u201cFacts are not science \u2014 as the dictionary is not literature.\u201d CHAPTER 1 \" Chemical Reactions Martin H. Fischer \u2014=} and Equations ie onsider the following situations of daily life and think what happens when \u2014 grapes get fermented. food is cooked. we respire. food gets digested in our body. milk is left at room temperature during summers. an iron tawa/pan/nail is left exposed to humid atmosphere. In all the above situations, the nature and the identity of the initial substance have somewhat changed. We have already learnt about physical and chemical changes of matter in our previous classes. Whenever a chemical change occurs, we can say that a chemical reaction has taken place. You may perhaps be wondering as to what is actually meant by a CAUTION: This Activity needs the teacher\u2019s assistance. It would be bet

In [ ]:
def annotate_dataset(dataset_dir):
    annotations = []
    for image_file in os.listdir(dataset_dir):
        if image_file.lower().endswith(".png"):
            # Extract page number from filename if possible (e.g., "jesc101_page_1.png")
            match = re.search(r'page_(\d+)', image_file)
            page_num = int(match.group(1)) if match else 0
            image_path = os.path.join(dataset_dir, image_file)
            ann = annotate_page(image_path, image_file, page_num)
            annotations.append(ann)
    return annotations


science_images_dir = '/content/drive/MyDrive/ScienceImages'
math_images_dir = '/content/drive/MyDrive/MathImages'

science_annotations = annotate_dataset(science_images_dir)
math_annotations = annotate_dataset(math_images_dir)


all_annotations = science_annotations + math_annotations
print(f"Total annotations generated: {len(all_annotations)}")



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Total annotations generated: 434


In [ ]:
output_path = '/content/drive/MyDrive/annotated_dataset_all.json'
with open(output_path, 'w') as outfile:
    json.dump(all_annotations, outfile, indent=4)
print(f"Annotations saved to {output_path}")


Annotations saved to /content/drive/MyDrive/annotated_dataset_all.json


In [ ]:
import os
from pdf2image import convert_from_path


science_pdf_dir = '/content/drive/MyDrive/science_dataset'
math_pdf_dir = '/content/drive/MyDrive/math_dataset'
output_science_dir = '/content/drive/MyDrive/ScienceImages'
output_math_dir = '/content/drive/MyDrive/MathImages'


os.makedirs(output_science_dir, exist_ok=True)
os.makedirs(output_math_dir, exist_ok=True)


def convert_pdfs_to_images(pdf_dir, output_dir):
    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, pdf_file)
            print(f"Converting {pdf_path} ...")
            pages = convert_from_path(pdf_path, dpi=200)
            for i, page in enumerate(pages, start=1):
                image_path = os.path.join(output_dir, f"{os.path.splitext(pdf_file)[0]}_page_{i}.png")
                page.save(image_path, "PNG")
                print(f"Saved: {image_path}")


convert_pdfs_to_images(science_pdf_dir, output_science_dir)


convert_pdfs_to_images(math_pdf_dir, output_math_dir)



Converting /content/drive/MyDrive/science_dataset/jesc101.pdf ...
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_1.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_2.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_3.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_4.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_5.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_6.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_7.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_8.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_9.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_10.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_11.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_12.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_13.png
Saved: /content/drive/MyDrive/ScienceImages/jesc101_page_14.png
Saved: /content/drive/MyDrive/ScienceImages/jes

In [ ]:
import os
def annotate_dataset(dataset_dir):
    annotations = []
    for image_file in os.listdir(dataset_dir):
        if image_file.lower().endswith(".png"):
            match = re.search(r'page_(\d+)', image_file)
            if match:
                page_num = int(match.group(1))
            else:
                page_num = 0
            image_path = os.path.join(dataset_dir, image_file)
            ann = annotate_page(image_path, image_file, page_num)
            annotations.append(ann)
    return annotations


science_images_dir = '/content/drive/MyDrive/ScienceImages'
math_images_dir = '/content/drive/MyDrive/MathImages'


science_annotations = annotate_dataset(science_images_dir)
math_annotations = annotate_dataset(math_images_dir)


all_annotations = science_annotations + math_annotations
print(f"Total annotations generated: {len(all_annotations)}")


Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 150, but your input_length is only 120. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 150, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Your max_length is set to 150, but your input_length is only 137. Since this is a summarization task, where outputs shorter than the inp

Total annotations generated: 434


In [ ]:
output_path = '/content/drive/MyDrive/annotated_dataset_all.json'
with open(output_path, 'w') as outfile:
    json.dump(all_annotations, outfile, indent=4)
print(f"Annotations saved to {output_path}")


Annotations saved to /content/drive/MyDrive/annotated_dataset_all.json
